In [2]:
import pandas as pd

In [3]:
# Load the cleaned data
df = pd.read_csv('clean_df.csv')

# Display the first few rows of the DataFrame to verify it's loaded correctly
print(df.head())

   quantity  unit         ingredient  ingredient_step  recipe_id
0      45.0    ml           dark rum                0          0
1      22.5    ml         lime juice                1          0
2      15.0    ml              sugar                2          0
3       1.0  dash  angostura bitters                3          0
4       6.0  drop             pernod                4          0
